In [1]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]
numeric_predictors = ['color', 'elo', 'opp_elo',
                      'game_time', 'start_time', 'weekday', 'day']
X = df[numeric_predictors]
y = np.array(df['result'])

In [3]:
X.shape

(2024, 7)

In [4]:
y.shape

(2024,)

In [5]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [6]:
X_train = X[:1900]
y_train = y[:1900]
X_test = X[1900:]
y_test = y[1900:]

In [7]:
X_train.shape

(1900, 7)

In [8]:
X_test.shape

(124, 7)

In [9]:
y_train.shape

(1900,)

In [10]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [11]:
std_sclr.scale_

array([  0.5       ,  40.68084711,  93.882371  , 139.29686294,
         4.70915264,   1.89793894,   8.68767756])

In [384]:
def _classifier():
    classifier = Sequential()
    
    classifier.add(Dense(units=7, kernel_initializer='uniform',
                         activation='relu', input_dim=7))
    
    classifier.add(
        Dense(units=14, kernel_initializer='uniform', activation='softsign'))

    classifier.add(
        Dense(units=7, kernel_initializer='uniform', activation='relu'))
    
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [385]:
classifier = _classifier()

In [386]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
1900/1900 [==============================] - 2s 1ms/step - loss: 0.6788 - acc: 0.5447
Epoch 2/100
1900/1900 [==============================] - 1s 317us/step - loss: 0.5770 - acc: 0.7163
Epoch 3/100
1900/1900 [==============================] - 1s 306us/step - loss: 0.5679 - acc: 0.7037
Epoch 4/100
1900/1900 [==============================] - 1s 319us/step - loss: 0.5639 - acc: 0.7142
Epoch 5/100
1900/1900 [==============================] - 1s 325us/step - loss: 0.5608 - acc: 0.7116
Epoch 6/100
1900/1900 [==============================] - 1s 327us/step - loss: 0.5619 - acc: 0.7221
Epoch 7/100
1900/1900 [==============================] - 1s 316us/step - loss: 0.5604 - acc: 0.7168
Epoch 8/100
1900/1900 [==============================] - 1s 330us/step - loss: 0.5604 - acc: 0.7174
Epoch 9/100
1900/1900 [==============================] - 1s 311us/step - loss: 0.5594 - acc: 0.7147
Epoch 10/100
1900/1900 [==============================] - 1s 309us/step - loss: 0.5594 - acc: 0.7189
E

In [387]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [388]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[50, 17],
       [20, 37]])

In [389]:
print(f'Accuracy = {(50+37)/(50+17+20+37)*100}%')

Accuracy = 70.16129032258065%


### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [128]:
classifier = KerasClassifier(
    build_fn=_classifier, batch_size=13, epochs=175)

In [390]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-4)

In [ ]:
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [12]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=7, kernel_initializer='uniform',
                         activation='relu', input_dim=7))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=28, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=4, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [13]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [14]:
parameters = {'batch_size': [10,20,30], 'nb_epoch': [100, 150, 200],
              'optimizer': ['Nadam', 'Adadelta', 'adamax']}

In [15]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='accuracy', cv=5, n_jobs=-4)

In [16]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
1520/1520 [==============================] - 1s 787us/step - loss: 0.6791 - acc: 0.5533
Epoch 1/1
1520/1520 [==============================] - 1s 795us/step - loss: 0.6797 - acc: 0.5711
Epoch 1/1
1520/1520 [==============================] - 1s 787us/step - loss: 0.6933 - acc: 0.4849
Epoch 1/1
1520/1520 [==============================] - 1s 830us/step - loss: 0.6845 - acc: 0.5618
Epoch 1/1
1520/1520 [==============================] - 1s 929us/step - loss: 0.6765 - acc: 0.5250
Epoch 1/1
1520/1520 [==============================] - 1s 853us/step - loss: 0.6932 - acc: 0.4862
Epoch 1/1
1520/1520 [==============================] - 1s 852us/step - loss: 0.6932 - acc: 0.5033
Epoch 1/1
1520/1520 [==============================] - 1s 887us/step - loss: 0.6932 - acc: 0.4829
Epoch 1/1
1520/1520 [==============================] - 1s 919us/step - loss: 0.6932 - acc: 0.4868
Epoch 1/1
1520/1520 [==============================] - 1s 932us/step - loss: 0.6932 - acc: 0.5086
Epoch 1/1
1520/1520 

1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.5007
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.5020
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6931 - acc: 0.5039
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4974
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6931 - acc: 0.5099
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4829
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4882
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6932 - acc: 0.4849
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6932 - acc: 0.4947
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6930 - acc: 0.5072
Epoch 1/1
1520/1520 [=============================

In [17]:
best_params = grid_search.best_params_
best_params

{'batch_size': 10, 'nb_epoch': 200, 'optimizer': 'Nadam'}

In [18]:
best_accu = grid_search.best_score_
best_accu

0.6163157894736843

In [19]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [20]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[53, 14],
       [19, 38]])

In [21]:
(56+38)/(67+57)

0.7580645161290323